In [ ]:
# TODO: apply GA for best rules selection or membership function creation

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from db import init
from sklearn.preprocessing import MinMaxScaler

In [2]:
db = init()
q = "CREATE TABLE IF NOT EXISTS drugs_info (id int, name text, amount int)"
db.query(q,[])

# for i in range(50):
#     q = "INSERT INTO drugs_info VALUES (?, ?, ?)"
#     v = [i, f'cit_{i}', np.random.randint(101)]
#     db.query(q,v)

q  = "select * from drugs_info"
drugs = db.query(q,[])
drugs.fetchall()

[(0, 'cit_0', 66),
 (1, 'cit_1', 63),
 (2, 'cit_2', 60),
 (3, 'cit_3', 66),
 (4, 'cit_4', 58),
 (5, 'cit_5', 82),
 (6, 'cit_6', 64),
 (7, 'cit_7', 70),
 (8, 'cit_8', 24),
 (9, 'cit_9', 19),
 (10, 'cit_10', 40),
 (11, 'cit_11', 0),
 (12, 'cit_12', 36),
 (13, 'cit_13', 43),
 (14, 'cit_14', 68),
 (15, 'cit_15', 13),
 (16, 'cit_16', 65),
 (17, 'cit_17', 23),
 (18, 'cit_18', 28),
 (19, 'cit_19', 16),
 (20, 'cit_20', 71),
 (21, 'cit_21', 29),
 (22, 'cit_22', 62),
 (23, 'cit_23', 80),
 (24, 'cit_24', 1),
 (25, 'cit_25', 29),
 (26, 'cit_26', 64),
 (27, 'cit_27', 70),
 (28, 'cit_28', 17),
 (29, 'cit_29', 79),
 (30, 'cit_30', 13),
 (31, 'cit_31', 54),
 (32, 'cit_32', 98),
 (33, 'cit_33', 81),
 (34, 'cit_34', 99),
 (35, 'cit_35', 80),
 (36, 'cit_36', 61),
 (37, 'cit_37', 61),
 (38, 'cit_38', 20),
 (39, 'cit_39', 67),
 (40, 'cit_40', 48),
 (41, 'cit_41', 68),
 (42, 'cit_42', 58),
 (43, 'cit_43', 79),
 (44, 'cit_44', 32),
 (45, 'cit_45', 47),
 (46, 'cit_46', 66),
 (47, 'cit_47', 81),
 (48, 'cit_48'

In [12]:

# ------------------------------
# membership function generator
# ------------------------------


# code from :
# https://github.com/scikit-fuzzy/scikit-fuzzy/blob/eecf303b701e3efacdc9b9066207ef605d4facaa/skfuzzy/membership/generatemf.py#L414 


def trimf(x, abc):
    assert len(abc) == 3, 'abc parameter must have exactly three elements.'
    a, b, c = np.r_[abc]     # Zero-indexing in Python
    assert a <= b and b <= c, 'abc requires the three elements a <= b <= c.'

    y = np.zeros(len(x))

    # Left side
    if a != b:
        idx = np.nonzero(np.logical_and(a < x, x < b))[0]
        y[idx] = (x[idx] - a) / float(b - a)

    # Right side
    if b != c:
        idx = np.nonzero(np.logical_and(b < x, x < c))[0]
        y[idx] = (c - x[idx]) / float(c - b)

    idx = np.nonzero(x == b)
    y[idx] = 1
    return y

In [5]:

# ------------------
# normalizing ranges
# ------------------

scaler = MinMaxScaler()
universe = scaler.fit_transform(np.arange(0, 101, 1).reshape(-1, 1)).reshape(1, -1)[0]


universe[45:101] # high

array([0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [57]:

# --------------------------
# Antecedent - current drugs
# --------------------------

X = {"low": None, "medium": None, "high": None}

X_ranges = [
                {"universe": universe[0:36], "mf": None},  # low
                {"universe": universe[30:51], "mf": None}, # medium
                {"universe": universe[45:101], "mf": None} # high
           ]


# generating membership functions
X_ranges[0]["mf"] = trimf(X_ranges[0]["universe"], [0, 0, 50]) # generating mf for low fuzzy set
X_ranges[1]["mf"] = trimf(X_ranges[1]["universe"], [0, 50, 100]) # generating mf for medium fuzzy set
X_ranges[2]["mf"] = trimf(X_ranges[2]["universe"], [50, 100, 100]) # generating mf for high fuzzy set



X["low"]    = X_ranges[0]
X["medium"] = X_ranges[1]
X["high"]   = X_ranges[2]



In [46]:

# -------------------------------------------
# Antecedent - drug sales over the past month
# -------------------------------------------

Y = {"very_few": None, "few": None, "medium": None, "many": None, "alot": None}

Y_ranges = [
                {"universe": universe[0:21], "mf": None},  # very_few
                {"universe": universe[15:36], "mf": None}, # few
                {"universe": universe[30:51], "mf": None}, # medium
                {"universe": universe[45:66], "mf": None}, # many
                {"universe": universe[60:101], "mf": None} # alot
           ]




# generating membership functions
Y_ranges[0]["mf"] = trimf(Y_ranges[0]["universe"], [0, 0, 10]) # generating mf for very_few fuzzy set
Y_ranges[1]["mf"] = trimf(Y_ranges[1]["universe"], [10, 20, 35]) # generating mf for few fuzzy set
Y_ranges[2]["mf"] = trimf(Y_ranges[2]["universe"], [20, 35, 45]) # generating mf for medium fuzzy set
Y_ranges[3]["mf"] = trimf(Y_ranges[3]["universe"], [35, 45, 55]) # generating mf for many fuzzy set
Y_ranges[4]["mf"] = trimf(Y_ranges[4]["universe"], [45, 55, 100]) # generating mf for alot fuzzy set



Y["very_few"] = Y_ranges[0]
Y["few"]      = Y_ranges[1]
Y["medium"]   = Y_ranges[2]
Y["many"]     = Y_ranges[3]
Y["alot"]     = Y_ranges[4]



In [52]:

# -------------------------------------------------------------
# Antecedent - drug sales over the past years in current season
# -------------------------------------------------------------

Z = {"very_few": None, "few": None, "medium": None, "many": None, "alot": None}

Z_ranges = [
                {"universe": universe[0:21], "mf": None},  # very_few
                {"universe": universe[15:36], "mf": None}, # few
                {"universe": universe[30:51], "mf": None}, # medium
                {"universe": universe[45:66], "mf": None}, # many
                {"universe": universe[60:101], "mf": None} # alot
           ]




# generating membership functions
Z_ranges[0]["mf"] = trimf(Z_ranges[0]["universe"], [0, 0, 10]) # generating mf for very_few fuzzy set
Z_ranges[1]["mf"] = trimf(Z_ranges[1]["universe"], [10, 20, 35]) # generating mf for few fuzzy set
Z_ranges[2]["mf"] = trimf(Z_ranges[2]["universe"], [20, 35, 45]) # generating mf for medium fuzzy set
Z_ranges[3]["mf"] = trimf(Z_ranges[3]["universe"], [35, 45, 55]) # generating mf for many fuzzy set
Z_ranges[4]["mf"] = trimf(Z_ranges[4]["universe"], [55, 100, 100]) # generating mf for alot fuzzy set



Z["very_few"] = Z_ranges[0]
Z["few"]      = Z_ranges[1]
Z["medium"]   = Z_ranges[2]
Z["many"]     = Z_ranges[3]
Z["alot"]     = Z_ranges[4]


In [43]:
# ----------------------------
# Consequent - amount of order
# ----------------------------

W = {"low": None, "medium": None, "high": None}

W_ranges = [
                {"universe": universe[0:36], "mf": None},  # low
                {"universe": universe[30:51], "mf": None}, # medium
                {"universe": universe[45:101], "mf": None} # high
           ]


# generating membership functions
W_ranges[0]["mf"] = trimf(W_ranges[0]["universe"], [0, 0, 50]) # generating mf for low fuzzy set
W_ranges[1]["mf"] = trimf(W_ranges[1]["universe"], [0, 50, 100]) # generating mf for medium fuzzy set
W_ranges[2]["mf"] = trimf(W_ranges[2]["universe"], [50, 100, 100]) # generating mf for high fuzzy set



W["low"]    = W_ranges[0]
W["medium"] = W_ranges[1]
W["high"]   = W_ranges[2]




In [41]:
class Rule:
    def __init__(self, left_c=None, operator="and", right_c=None, _then=None):
        self.left_c   = left_c
        self.operator = operator
        self.right_c  = right_c
        self._then = _then
        
        

In [56]:
# ---------------------------------
# sales over the past month rules
month_rules = []
# ---------------------------------

month_rules.append(Rule(left_c=Y["alot"], operator="and", right_c=X["low"],    _then="high"))
month_rules.append(Rule(left_c=Y["alot"], operator="and", right_c=X["medium"], _then="high"))
month_rules.append(Rule(left_c=Y["alot"], operator="and", right_c=X["high"],   _then="high"))

month_rules.append(Rule(left_c=Y["many"], operator="and", right_c=X["low"],    _then="high"))
month_rules.append(Rule(left_c=Y["many"], operator="and", right_c=X["medium"], _then="high"))
month_rules.append(Rule(left_c=Y["many"], operator="and", right_c=X["high"],   _then="medium"))

month_rules.append(Rule(left_c=Y["medium"], operator="and", right_c=X["low"],    _then="high"))
month_rules.append(Rule(left_c=Y["medium"], operator="and", right_c=X["medium"], _then="medium"))
month_rules.append(Rule(left_c=Y["medium"], operator="and", right_c=X["high"],   _then="low"))

month_rules.append(Rule(left_c=Y["few"], operator="and", right_c=X["low"],    _then="medium"))
month_rules.append(Rule(left_c=Y["few"], operator="and", right_c=X["medium"], _then="low"))
month_rules.append(Rule(left_c=Y["few"], operator="and", right_c=X["high"],   _then="low"))

month_rules.append(Rule(left_c=Y["very_few"], operator="and", right_c=X["low"],    _then="low"))
month_rules.append(Rule(left_c=Y["very_few"], operator="and", right_c=X["medium"], _then="low"))
month_rules.append(Rule(left_c=Y["very_few"], operator="and", right_c=X["high"],   _then="low"))



In [127]:
# ---------------------------------------------------
# Rules - sales over the past years in current season
years_rules = []
# ---------------------------------

years_rules.append(Rule(left_c=Y["alot"], operator="and", right_c=X["low"],    _then="high"))
years_rules.append(Rule(left_c=Y["alot"], operator="and", right_c=X["medium"], _then="high"))
years_rules.append(Rule(left_c=Y["alot"], operator="and", right_c=X["high"],   _then="high"))

years_rules.append(Rule(left_c=Y["many"], operator="and", right_c=X["low"],    _then="high"))
years_rules.append(Rule(left_c=Y["many"], operator="and", right_c=X["medium"], _then="high"))
years_rules.append(Rule(left_c=Y["many"], operator="and", right_c=X["high"],   _then="medium"))

years_rules.append(Rule(left_c=Y["medium"], operator="and", right_c=X["low"],    _then="high"))
years_rules.append(Rule(left_c=Y["medium"], operator="and", right_c=X["medium"], _then="medium"))
years_rules.append(Rule(left_c=Y["medium"], operator="and", right_c=X["high"],   _then="low"))

years_rules.append(Rule(left_c=Y["few"], operator="and", right_c=X["low"],    _then="medium"))
years_rules.append(Rule(left_c=Y["few"], operator="and", right_c=X["medium"], _then="low"))
years_rules.append(Rule(left_c=Y["few"], operator="and", right_c=X["high"],   _then="low"))

years_rules.append(Rule(left_c=Y["very_few"], operator="and", right_c=X["low"],    _then="low"))
years_rules.append(Rule(left_c=Y["very_few"], operator="and", right_c=X["medium"], _then="low"))
years_rules.append(Rule(left_c=Y["very_few"], operator="and", right_c=X["high"],   _then="low"))



In [58]:
# -----------------------
# Defuzzification process
# -----------------------